## FashionMNIST Attack model
This notebook contains code for a CNN classifier on FashionMNIST dataset.

In [2]:
import torch
from torchvision import datasets, transforms

loader = torch.utils.data.DataLoader(datasets.FashionMNIST("data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor()
    ])), batch_size=60000, shuffle=False)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
data = next(iter(loader))

In [4]:
data[0].shape

torch.Size([60000, 1, 28, 28])

In [5]:
std, mean = torch.std_mean(data[0])

In [6]:
std, mean

(tensor(0.3530), tensor(0.2860))

In [1]:
FMNIST_MEAN = 0.2860
FMNIST_STD = 0.3530

In [2]:
import torch
from torchvision import datasets, transforms

train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST("data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((FMNIST_MEAN,), (FMNIST_STD,))
    ])), batch_size=128, shuffle=False)
val_loader = torch.utils.data.DataLoader(datasets.FashionMNIST("data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((FMNIST_MEAN,), (FMNIST_STD,))
    ])), batch_size=128, shuffle=False)

In [3]:
import pytorch_lightning as pl
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from optimizee.mnist import FMnistConvModel

class FMNISTClassifier(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.model = FMnistConvModel()
        self.criterion = nn.NLLLoss()
        self.metrics = {
            "accuracy": {
                "train": torchmetrics.Accuracy(),
                "val": torchmetrics.Accuracy()
            }
        }

    def step(self, batch, step_name="train"):
        X, y = batch
        outputs = self.model(X)
        loss = self.criterion(outputs, y)
        preds = self.forward(X)
        metric = self.metrics["accuracy"][step_name]
        metric.update(preds.cpu(), y.cpu())
        metric_val = metric.compute()
        self.log(f"{step_name}_loss", loss, on_epoch=True)
        self.log(f"{step_name}_accuracy", metric_val, on_epoch=True)
        return loss

    def forward(self, X, *args):
        return self.model(X)

    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")

    def predict_step(self, batch, batch_idx):
        X, y = batch
        return self.forward(X)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.hparams.lr)
        return optimizer

In [5]:
import wandb
from pytorch_lightning.loggers import WandbLogger

NUM_EPOCHS = 10

wandb_logger = WandbLogger(project="optml-project", name=f"fmnist-classifier")

model = FMNISTClassifier(lr=1e-4)
trainer = pl.Trainer(default_root_dir="models/fmnist", max_epochs=NUM_EPOCHS, logger=wandb_logger, accelerator="gpu")
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
wandb.finish()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type            | Params
----------------------------------------------
0 | model     | FMnistConvModel | 431 K 
1 | criterion | NLLLoss         | 0     
----------------------------------------------
431 K     Trainable params
0         Non-trainable params
431 K     Total params
1.724     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/root/.conda/envs/optml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/root/.conda/envs/optml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_accuracy_epoch,▁▅▆▆▇▇▇███
train_accuracy_step,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
train_loss_epoch,█▄▃▂▂▂▁▁▁▁
train_loss_step,█▅▃▄▄▃▃▄▃▃▃▂▃▃▂▂▂▃▂▂▂▂▁▂▁▂▂▁▂▂▂▂▁▂▂▂▁▁▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▄▅▆▇▇▇███
val_loss,█▆▄▃▃▂▂▁▁▁
epoch,9
train_accuracy_epoch,0.85532
train_accuracy_step,0.85719


Save the model

In [6]:
from collections import OrderedDict
fmnist_model_dict = OrderedDict({name.replace("model.", ""): parameter for name, parameter in model.state_dict().items()})
torch.save(fmnist_model_dict, "ckpt/attack_model/fmnist_cnn.pt")

Predict and save indices for correctly classified examples

In [14]:
model = FMNISTClassifier(lr=1e-4)
model.model.load_state_dict(torch.load("ckpt/attack_model/fmnist_cnn.pt"))
preds = trainer.predict(model, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/.conda/envs/optml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 469it [00:00, ?it/s]

In [15]:
preds = torch.cat(preds)

In [16]:
preds.shape

torch.Size([10000, 10])

In [17]:
preds = preds.argmax(dim=1)

In [18]:
val_targets = []

for _, y in val_loader:
    val_targets.append(y)

val_targets = torch.cat(val_targets)

In [19]:
val_targets.shape

torch.Size([10000])

In [20]:
(preds == val_targets).sum() / len(val_targets)

tensor(0.8772)

In [21]:
correct_indices = torch.where(preds == val_targets)[0]

In [22]:
correct_indices

tensor([   0,    1,    2,  ..., 9996, 9997, 9998])

In [24]:
import numpy as np

with open("data/fmnist_correct/label_correct_index.npy", "wb") as f:
    np.save(f, correct_indices.numpy())